## Target Function Extraction

### Implementation1

In [22]:
import re
from pprint import pprint

def extract_between(text, start, end, extend=False, multiple=False):
    target = r'(.*)' if extend else r'(.*?)'
    pattern = re.escape(start) + target + re.escape(end)
    matches = re.findall(pattern, text)

    if multiple:
        results = []
        for match in matches:
            results.append(match.strip())
        return results
    else:
        # Return the first match if available, else None
        return matches[0].strip() if matches else None

# Example usage
text = """
@converter(torch.sin, torch.Tensor.sin, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def converter_sin(input, *args, **kwargs):
    def func(input, *args, **kwargs):
        return tf.math.sin(input)
    return func


@converter(torch.cos, torch.Tensor.cos, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def converter_cos(input, *args, **kwargs):
    def func(input, *args, **kwargs):
        return tf.math.cos(input)
    return func


@converter(torch.add, torch.Tensor.add, torch.Tensor.add_, torch.Tensor.__add__, torch.Tensor.__iadd__, torch.Tensor.__radd__, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS_OR_PYTORCH, autocast=True)
def converter_add(input, other, *args, **kwargs):
    def func(input, other, *args, **kwargs):
        return input + other
    return func
"""


### Tests

#### end = ', channel_ordering_strategy'

It makes the `extend` argument useless, but free from the additional arguments to the target functions.

In [23]:
# Test the function
start = '@converter('
end = ', channel_ordering_strategy'

bools = [True, False]

for _bool_ex in bools:
	extend = _bool_ex
	for _bool_mul in bools:
		multiple = _bool_mul  
		print("extend:", str(_bool_ex), "multiple: ", str(_bool_mul))
		pprint(extract_between(text, start, end, extend=extend, multiple=multiple))
		print("\n")


extend: True multiple:  True
['torch.sin, torch.Tensor.sin',
 'torch.cos, torch.Tensor.cos',
 'torch.add, torch.Tensor.add, torch.Tensor.add_, torch.Tensor.__add__, '
 'torch.Tensor.__iadd__, torch.Tensor.__radd__']


extend: True multiple:  False
'torch.sin, torch.Tensor.sin'


extend: False multiple:  True
['torch.sin, torch.Tensor.sin',
 'torch.cos, torch.Tensor.cos',
 'torch.add, torch.Tensor.add, torch.Tensor.add_, torch.Tensor.__add__, '
 'torch.Tensor.__iadd__, torch.Tensor.__radd__']


extend: False multiple:  False
'torch.sin, torch.Tensor.sin'




#### end = ','

It makes the `extend` argument functional, but it can include non-target results.

In [24]:
# Test the function
start = '@converter('
end = ','

bools = [True, False]

for _bool_ex in bools:
	extend = _bool_ex
	for _bool_mul in bools:
		multiple = _bool_mul  
		print("extend:", str(_bool_ex), "multiple: ", str(_bool_mul))
		pprint(extract_between(text, start, end, extend=extend, multiple=multiple))
		print("\n")

extend: True multiple:  True
['torch.sin, torch.Tensor.sin',
 'torch.cos, torch.Tensor.cos',
 'torch.add, torch.Tensor.add, torch.Tensor.add_, torch.Tensor.__add__, '
 'torch.Tensor.__iadd__, torch.Tensor.__radd__, '
 'channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS_OR_PYTORCH']


extend: True multiple:  False
'torch.sin, torch.Tensor.sin'


extend: False multiple:  True
['torch.sin', 'torch.cos', 'torch.add']


extend: False multiple:  False
'torch.sin'




## Implementation2

- discard `extend` argument
- use out[0] for single output instead
- split each result as `list(str)`


In [42]:
import re

def extract_between(text, start, end, multiple=False):
    target = r'(.*?)'
    pattern = re.escape(start) + target + re.escape(end)
    matches = re.findall(pattern, text)

    if multiple:
        results = []
        for match in matches:
            results.append([item.strip() for item in match.strip().split(',')])
        return results
    else:
        if matches:
            return [item.strip() for item in matches[0].strip().split(',')]
        else:
            return None

In [43]:
# Test the function
start = '@converter('
end = ', channel_ordering_strategy'

bools = [True, False]

for _bool_mul in bools:
	multiple = _bool_mul  
	print("multiple: ", str(_bool_mul))
	pprint(extract_between(text, start, end, multiple=multiple))
	print("\n")


multiple:  True
[['torch.sin', 'torch.Tensor.sin'],
 ['torch.cos', 'torch.Tensor.cos'],
 ['torch.add',
  'torch.Tensor.add',
  'torch.Tensor.add_',
  'torch.Tensor.__add__',
  'torch.Tensor.__iadd__',
  'torch.Tensor.__radd__']]


multiple:  False
['torch.sin', 'torch.Tensor.sin']




## `__doc__` Extration

In [44]:
# Example usage
text = """
@converter(torch.sin, torch.Tensor.sin, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def converter_sin(input, *args, **kwargs):
    def func(input, *args, **kwargs):
        return tf.math.sin(input)
    return func


@converter(torch.cos, torch.Tensor.cos, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def converter_cos(input, *args, **kwargs):
    def func(input, *args, **kwargs):
        return tf.math.cos(input)
    return func


@converter(torch.add, torch.Tensor.add, torch.Tensor.add_, torch.Tensor.__add__, torch.Tensor.__iadd__, torch.Tensor.__radd__, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS_OR_PYTORCH, autocast=True)
def converter_add(input, other, *args, **kwargs):
    def func(input, other, *args, **kwargs):
        return input + other
    return func

@converter(nn.Linear, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def converter_Linear(self, input: Tensor):
    out_filters, in_filters = self.weight.shape
    weights = self.weight.cpu().detach().numpy()
    weights = weights.transpose(1, 0)

    use_bias = self.bias is not None
    if use_bias:
        biases = self.bias.cpu().detach().numpy()
        params = [weights, biases]
    else:
        params = [weights]
    return keras.layers.Dense(out_filters, use_bias=use_bias, weights=params)
"""

### Implementation1

In [46]:
targets = extract_between(text, start, end, multiple=True)
targets

[['torch.sin', 'torch.Tensor.sin'],
 ['torch.cos', 'torch.Tensor.cos'],
 ['torch.add',
  'torch.Tensor.add',
  'torch.Tensor.add_',
  'torch.Tensor.__add__',
  'torch.Tensor.__iadd__',
  'torch.Tensor.__radd__'],
 ['nn.Linear']]